<a href="https://colab.research.google.com/github/m-newhauser/fine-tune-qwen3/blob/main/Gemma3_(4B)_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inference Gemma (base) with LoRA adapters pre-trained on Growth content

### Installation

In [1]:
# # This used to work but now it doesn't
# %%capture
# !pip install pip3-autoremove
# !pip-autoremove torch torchvision torchaudio -y
# !pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu121
# !pip install unsloth
# !pip install evaluate

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [2]:
import torch
import datasets
import pandas as pd
import numpy as np

from unsloth import FastModel
from peft import PeftModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
from google.colab import userdata

# Vars
HF_TOKEN = userdata.get('HF_TOKEN') # Replace this with my HuggingFace Token (1Password > DevGrowth > HF_TOKEN_MN)
BASE_MODEL_PATH = "unsloth/gemma-3-4b-it"
LORA_MODEL_PATH = "m-newhauser/gemma-3-lora"

## Load the base model and adapters

In [4]:
# !rm -rf unsloth_compiled_cache
# !pip cache purge

In [5]:
# First load the base model and tokenzier
model, tokenizer = FastModel.from_pretrained(
    model_name = BASE_MODEL_PATH,
    max_seq_length = 5000, # Choose any for long context!
    # max_seq_length = 2048, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
)

==((====))==  Unsloth 2025.5.6: Fast Gemma3 patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.


model.safetensors:   0%|          | 0.00/4.56G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

processor_config.json:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

In [6]:
# Load pre-trained LoRA adapters and apply them to base model
model = PeftModel.from_pretrained(model, LORA_MODEL_PATH)

adapter_config.json:   0%|          | 0.00/1.10k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/59.7M [00:00<?, ?B/s]

## Perform inference
### Basic social media post

In [24]:
# Specify maximum post length
max_post_length = 1000

Don't edit this cell! 👇 Run the cell first and then enter your prompt in the input box :)

In [25]:
# Enter your prompt
prompt = input("Type your prompt here: ")
print(f"You entered: {prompt}")

Type your prompt here: Explain the 5 levels of AI agents for people who are not familiar with AI or agents. Your tone and word choice should be approachable and easy-to-understand. You can use appropriate technical terminology but make sure that it's understandable. Include the key components of agents (memory, tools, reasoning, and permissions. Also reference the different levels of autonomy.
You entered: Explain the 5 levels of AI agents for people who are not familiar with AI or agents. Your tone and word choice should be approachable and easy-to-understand. You can use appropriate technical terminology but make sure that it's understandable. Include the key components of agents (memory, tools, reasoning, and permissions. Also reference the different levels of autonomy.


Same thing here

In [26]:
# Optionally add context for the model to reference
context = input("Add context here: ")

Add context here: AI agents can be categorized into different levels of sophistication and autonomy, ranging from simple reflex agents to complex, goal-based and multi-agent systems. These levels are based on the agent's ability to perceive its environment, plan actions, and achieve goals.  Levels of AI Agents: Reactive Agents: These agents react directly to their environment without any prior planning or memory. They respond to changes in their surroundings and make decisions based on immediate sensory input. Examples include simple reflex agents. Model-Based Agents: These agents build a model of their environment and use it to predict future states. They can plan actions based on this model and consider the consequences of their choices. This level is also known as utility-based agents. Goal-Based Agents: These agents have specific goals they strive to achieve and plan their actions to reach these goals. They can also adapt their plans based on the information they receive from the e

In [27]:
# Clear old messages
messages = []

# Place prompt into prompt instructions
prompt_instructions = f"""
    Write a professional social media post based on this short sentence summary:
    {prompt}
    You should always use the summary as a guideline for the post and do not stray from it unless it is inevitable.
    You should also use the context to include relevant technical content in the post.
    Your written post should pay close attention to the following set of instructions related to writing style, tone, format and organisation:
    - Start with a genuinely engaging or provocative hook that lures the reader in (avoid overly generic enthusiasm like 'Exciting times!' or 'Thrilled to share!').
    - Present your key point early and make sure it is clear and relevant.
    - When relevant, use bullet points or emojis to break up the text in a way that enhances readability and understanding, not just as superficial decorations (avoid simply adding random emojis like ✨ or 🚀 without clear purpose).
    - Use a friendly and conversational tone whilst staying professional (avoid sounding overly formal or detached like a generic AI announcement).
    - Avoid jargon and overly-technical language, explaining concepts clearly and concisely.
    - Use short paragraphs and sentences to improve readability.
    - Use emojis and line breaks only where it makes sense to enhance the message and visual flow.

    Do not emulate common characteristics of poorly written AI-generated social media posts, such as:
    - Using overly generic and enthusiastic language without specific context (e.g., 'Fascinating exploration in AI!').
    - Focusing on abstract technical concepts without connecting them to real-world value or user benefits.
    - Employing a predictable and excessive set of trendy AI/tech hashtags without adding unique or context-specific tags (e.g., just #AI #MachineLearning #Innovation).
    - Sounding impersonal and lacking any unique perspective or human voice.
    - Adopting a stilted or slightly formal tone, even when trying to be casual (e.g., 'Delving into the intricacies of neural networks...').
    - Over-relying on common LLM vocabulary in a generic way (e.g., 'Witnessing the transformative potential...').
    - Including emojis superficially without genuinely enhancing the message.
    - Using language and expressions common among generative LLMs like 'AI enthusiasts', 'delve into', 'demystify', 'game-changer', 'level up', or 🚀.
    - Do not repeat words and phrases throughout the post.

    Treat it as plain text (with emojis, bullet points, lists, etc).
    You can use markdown bold (**) and italics (*) _sparingly_ to emphasize key points where it naturally enhances understanding, not just for generic emphasis. Use any examples (if given) as a reference point for effective communication.
"""

# Format message history for the conversation
if context:
  messages = [
      {"role": "user", "content": [{"type": "text", "text": "Here is some context to use for the following task:"}]},
      {"role": "assistant", "content": [{"type": "text", "text": context}]},
      {"role": "user", "content": [{"type": "text", "text": prompt_instructions}]},
  ]
else:
  messages = [
      {"role": "user",
       "content": [{"type" : "text", "text" : prompt_instructions,}]}
  ]

In [29]:
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)

# Generate the initial response
from transformers import TextStreamer
output = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = max_post_length, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0,
    top_p = 0.95,
    top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

initial_response = tokenizer.decode(output[0], skip_special_tokens=True)

A lot of new information about AI agents is spreading around that can be a little overwhelming. Here's a breakdown of the 5 levels of AI agents that will help you understand what's going on.

Let's define the 5 levels of AI Agents:
1. Reflex agents: These agents react directly to changes in the environment and make decisions based on immediate sensory input.
2. Model-based agents: These agents build a model of their environment and use it to predict future states.
3. Goal-based agents: These agents have specific goals they strive to achieve and plan their actions to reach these goals.
4. Multi-agent systems: These systems involve multiple agents working together to achieve a common goal.
5. Autonomous agents: These agents have a high degree of autonomy and can make decisions without human intervention.

We can also define the different levels of autonomy within this framework and explain why you might want to invest in a specific approach based on the need.

Key components of AI agents

Run the following cell and paste the output into the `original_prompt` column in the [Prompt Revision Google Sheet](https://docs.google.com/spreadsheets/d/12JpebB19Otn0MlQBdVvCrpk0hnNsVzw5HNhHYoXoNYU/edit?usp=sharing).

In [ ]:
# print(prompt_instructions)

Run the following cell and paste the output into the `original_output` column in the [Prompt Revision Google Sheet](https://docs.google.com/spreadsheets/d/12JpebB19Otn0MlQBdVvCrpk0hnNsVzw5HNhHYoXoNYU/edit?usp=sharing).

In [ ]:
# print(initial_response)

### Revision prompting

If you're not happy with the model's initial response, you can add instructions to revise the prompt below.

Right now, the model is not great at revising posts when given specific instructions to do so!

So the goal is to train the model to do this better by generating a dataset that looks like this:

```
{
  "original_prompt": "The original prompt goes here.",
  "original_output": "The model's first response goes here.",
  "revision_prompt": "The prompt with the instructions for revision goes here.",
  "HUMAN_revised_output": "The HUMAN-revised (aka the final version of the post you'd actually post) post goes here."
}
```

In [34]:
# Enter follow up prompt
revision_prompt = input("Type your follow up here: ")
print(f"You entered: {revision_prompt}")

Type your follow up here: Start with a much more seductive or controversial hook about AI agents and autonomy. Follow up the hook with essential information that makes the reader want to read on. For each level agent, define the level in on succinct sentence and distinguish it with the appropriate number emoji. Then include all the relevant information about the levels including their use of the components I mentioned and their autonomy is short, succinct bullet points. End with my opinion that I think agents are early and autonomous agents are scary. Follow up saying that agents are promising but need further development and in the meantime we should give them limited permissions and autonomy and watch them closely. 
You entered: Start with a much more seductive or controversial hook about AI agents and autonomy. Follow up the hook with essential information that makes the reader want to read on. For each level agent, define the level in on succinct sentence and distinguish it with th

In [35]:
# Follow up prompt instructions
revision_prompt_instructions = f"""
    Revise the following social media post based on the user feedback provided.
    Keep the original post's core message and technical content intact unless the feedback explicitly suggests changes.

    **Original Post:**
    {initial_response}

    **User Feedback:**
    {revision_prompt}
    Pay close attention to the following instructions while revising:

    **Consider and address the user feedback directly and thoughtfully.**

    **Maintain the original post's professional yet friendly and conversational tone.**

    **Review and refine the post based on the initial writing style guidelines:**
    - Ensure the hook remains engaging or is improved based on feedback.
    - Verify the key point is still clear and relevant after revision.
    - Check if bullet points or emojis can be used more effectively to improve readability based on feedback.
    - Continue to avoid jargon and overly-technical language unless specifically requested by the feedback.
    - Keep paragraphs and sentences concise.
    - Use emojis and line breaks appropriately.

    **Continue to avoid common LLM expressions:** 'AI enthusiasts', 'delve into', 'demystify', 'game-changer', 'level up', or 🚀.

    **Avoid unnecessary repetition of words and phrases.**

    **Treat the output as plain text (with emojis, bullet points, lists, etc.).**

    **Use markdown bold (**) and italics (*) _sparingly_ only for emphasis if it improves clarity based on feedback.**

    Response ONLY with the text content for the post.
"""

# Append initial response and follow up prompt to message history
messages.append({"role": "assistant", "content": [{"type": "text", "text": initial_response}]})
messages.append({"role": "user", "content": [{"type": "text", "text": revision_prompt_instructions}]})

text_with_follow_up = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
)

output_with_follow_up = model.generate(
    **tokenizer([text_with_follow_up], return_tensors = "pt").to("cuda"),
    max_new_tokens = max_post_length,  # Adjust as needed for the follow-up
    temperature = 0.3,    # You might want to adjust generation parameters
    top_p = 0.7,
    top_k = 32,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

user
Here is some context to use for the following task:
model
AI agents can be categorized into different levels of sophistication and autonomy, ranging from simple reflex agents to complex, goal-based and multi-agent systems. These levels are based on the agent's ability to perceive its environment, plan actions, and achieve goals.  Levels of AI Agents: Reactive Agents: These agents react directly to their environment without any prior planning or memory. They respond to changes in their surroundings and make decisions based on immediate sensory input. Examples include simple reflex agents. Model-Based Agents: These agents build a model of their environment and use it to predict future states. They can plan actions based on this model and consider the consequences of their choices. This level is also known as utility-based agents. Goal-Based Agents: These agents have specific goals they strive to achieve and plan their actions to reach these goals. They can also adapt their plans bas

Run the following cell and paste the output into the `revision_prompt` column in the [Prompt Revision Google Sheet](https://docs.google.com/spreadsheets/d/12JpebB19Otn0MlQBdVvCrpk0hnNsVzw5HNhHYoXoNYU/edit?usp=sharing).

In [37]:
# print(revision_prompt_instructions)

"\n    Revise the following social media post based on the user feedback provided.\n    Keep the original post's core message and technical content intact unless the feedback explicitly suggests changes.\n\n    **Original Post:**\n    user\nHere is some context to use for the following task:\nmodel\nAI agents can be categorized into different levels of sophistication and autonomy, ranging from simple reflex agents to complex, goal-based and multi-agent systems. These levels are based on the agent's ability to perceive its environment, plan actions, and achieve goals.  Levels of AI Agents: Reactive Agents: These agents react directly to their environment without any prior planning or memory. They respond to changes in their surroundings and make decisions based on immediate sensory input. Examples include simple reflex agents. Model-Based Agents: These agents build a model of their environment and use it to predict future states. They can plan actions based on this model and consider th

A work in progress. Please ignore for now. 👇

### Post re-wording

In [12]:
original_text = """
The 5 Levels of AI Agents

Lets explore the 5 levels of AI Agents, from simple to complex. Always start with level 1 and add complexity as needed.

Level 1: Agent with tools and instructions. When people say agents are just LLM + tool calls in a loop, this is what they mean (this also tells you their level of understanding).

Instructions "teach" the Agent how to achieve its task and tools let Agents interact with external environments to push or pull data.

Level 2: Agent with knowledge and storage. Rarely does a model have all the information it needs to achieve its task and we obviously can't jam everything in the context, so we give the Agent knowledge that it searches at runtime (i.e Agentic RAG or Dynamic few-shot).

Knowledge search needs to be hybrid (full-text and semantic). Hybrid search + reranking is the best out-of-the-box Agentic Search strategy you can use.

Storage saves the Agent's state in a database. LLM calls are "stateless" and storage makes Agents "stateful" by storing messages in a database and adding them to the current call as needed.

Level 3: Agent with memory and reasoning. Memory let's an Agent remember details about a user and personalize its responses across sessions. This is a fairly new concept that everyone is still exploring. The part of memory im the most excited about is "self-learning", more on this soon.

Reasoning is a key feature that every agent builder should know when and how to use. It not only improves cognitive reasoning (understanding of data and instructions) but also improves the success rate of each step.

Level 4: Multi Agent Teams. Agents work best when they have a narrow scope (i.e. specialized to a domain) and a reasonably small set of tools (<10 ish). By putting agents together in a team, we can increase the overall capabilities and solve broader, more complex problems.

Remember to add reasoning, otherwise the Team leader struggles to work on complex tasks. My current (2025) belief is that autonomous multi-agent teams don't work. They work <1/2 the time and thats no good.

Agno comes with an industry leading multi-agent architecture that supports 3 modes of execution: coordinate, route and collaborate with the ability to automatically manage agentic memory and context.

Level 5: Agentic Systems. APIs (i.e. servers) that take in a request, asynchronously complete the task and stream back the result. These are hard, very hard - when the request comes in we need to save the state in a database, trigger an async job and stream the results back as they're ready. Websockets can work here, but they are not an each tech to work with.

Agentic systems is where the $ is and what everyone is trying to build. We've put out the Agent API, Agent UI and detailed documentation on how to build them. I hope to write more on this to help AI engineers build better systems.

If you liked reading this, please let me know and consider starring Agno: https://agno.link/gh
"""

original_text_len = len(original_text)

Original text has 503 words.


In [17]:
rewording_instructions = """
Start with a seductive, controversial, or other highly engaging hook about AI agents and autonomy.
Make the post less dense, using much shorter sentences and bullet points where necessary.
Assign an emoji to each level of AI agents and explain the level in one succint and clear sentence.
Then include anything else important about the level in bullet points. Include a real-life example
in the last bullet point for each level. Remove the author's personal opinions from the post.
Include my opinions on AI agents that right now I think they tend to "spiral" when they're given
too much autonomy. Also that giving agents too much autonomy risks major reputational damage
and has ethical concerns.
"""

In [22]:
# Clear old messages
messages = []

context = ""

prompt_instructions_rewording = f"""You are an expert social media content creator.
Your task is to ethically repurpose and reimagine existing social media content.
You will be given the original text and (optional) instructions for editing it.

Here is the original social media post:
{original_text}

Post specific instructions for editing:
{rewording_instructions}

Consider the following when rewriting:
* **Editing instructions:** Follow the editing instructions as closely as possible.
* **Tone:** Adjust the overall feeling and attitude of the post (e.g., reflective, insightful, sharing a lesson learned, offering a perspective, etc.).
* **Style:** Modify the writing style (e.g., professional yet approachable, concise and impactful, storytelling-based, thought-provoking questions, etc.).
* **Voice:** Emulate the typical voice and persona of the LinkedIn content your model is trained on.
* **Engagement:** Aim to create a post that resonates with a professional LinkedIn audience while reflecting the stylistic nuances of your training data.
* **Ethical Recycling:** Ensure the rewritten post offers a fresh perspective and isn't a direct copy, thus ethically reusing the core information.

Respond only with the post content and nothing else.
"""

# Format message history for the conversation
if context:
  messages = [
      {"role": "user", "content": [{"type": "text", "text": "Here is some context to use for the following task:"}]},
      {"role": "assistant", "content": [{"type": "text", "text": context}]},
      {"role": "user", "content": [{"type": "text", "text": prompt_instructions_rewording}]},
  ]
else:
  messages = [
      {"role": "user",
       "content": [{"type" : "text", "text" : prompt_instructions_rewording,}]}
  ]

In [23]:
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)

# Generate the initial response
from transformers import TextStreamer
output = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 2000, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 0.6,
    top_p = 0.8,
    top_k = 32,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

initial_response = tokenizer.decode(output[0], skip_special_tokens=True)

⚠️ 🚨 🚨 🚨 🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨<bos>